In [20]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Date    : 2017-12-25 16:41:30
# @Author  : guanglinzhou (xdzgl812@163.com)

import os
import numpy as np
import pandas as pd
import xgboost as xgb

project_path = '/Users/guanglinzhou/Desktop/DC_Mac/DC_Code/'
train_data = pd.read_csv(project_path + 'preprocess/traindata_output/train_data.csv')
test_data = pd.read_csv(project_path + 'preprocess/testdata_output/test_data.csv')


In [21]:
test_data.head()

,userid,actionType1,actionType2,actionType3,actionType4,actionType5,actionType6,actionType7,actionType8,actionType9,gender,province,age
0,100000000371,1,0,0,0,6,4,2,0,0,NaN,shanghai,NaN
1,100000001215,11,0,0,0,14,2,0,0,0,NaN,NaN,NaN
2,100000001445,83,0,0,0,66,14,0,7,4,M,zhejiang,NaN
3,100000001575,19,0,0,0,42,22,0,1,0,W,jiangsu,NaN
4,100000001601,7,0,1,0,1,2,0,2,2,NaN,shanghai,NaN


In [22]:
test_data_userid=test_data['userid']

In [3]:
train_data.head()

,userid,actionType1,actionType2,actionType3,actionType4,actionType5,actionType6,actionType7,actionType8,actionType9,gender,province,age,orderType
0,100000000013,79,1,4,1,32,18,1,4,3,M,NaN,50.0,0
1,100000000111,1,0,0,0,1,1,0,0,0,NaN,shanghai,NaN,0
2,100000000127,2,0,0,0,2,0,2,0,0,NaN,shanghai,NaN,0
3,100000000231,15,6,2,5,10,6,0,0,0,M,beijing,40.0,0
4,100000000379,42,3,7,6,14,11,1,0,0,M,beijing,NaN,0


In [4]:
test_data.head()

,userid,actionType1,actionType2,actionType3,actionType4,actionType5,actionType6,actionType7,actionType8,actionType9,gender,province,age
0,100000000371,1,0,0,0,6,4,2,0,0,NaN,shanghai,NaN
1,100000001215,11,0,0,0,14,2,0,0,0,NaN,NaN,NaN
2,100000001445,83,0,0,0,66,14,0,7,4,M,zhejiang,NaN
3,100000001575,19,0,0,0,42,22,0,1,0,W,jiangsu,NaN
4,100000001601,7,0,1,0,1,2,0,2,2,NaN,shanghai,NaN


In [13]:
train_data.iloc[:,1:-1].head()

,actionType1,actionType2,actionType3,actionType4,actionType5,actionType6,actionType7,actionType8,actionType9,gender,province,age
0,79,1,4,1,32,18,1,4,3,M,NaN,50.0
1,1,0,0,0,1,1,0,0,0,NaN,shanghai,NaN
2,2,0,0,0,2,0,2,0,0,NaN,shanghai,NaN
3,15,6,2,5,10,6,0,0,0,M,beijing,40.0
4,42,3,7,6,14,11,1,0,0,M,beijing,NaN


In [14]:
test_data=test_data.iloc[:,1:]

In [23]:
test_data.head()

,userid,actionType1,actionType2,actionType3,actionType4,actionType5,actionType6,actionType7,actionType8,actionType9,gender,province,age
0,100000000371,1,0,0,0,6,4,2,0,0,NaN,shanghai,NaN
1,100000001215,11,0,0,0,14,2,0,0,0,NaN,NaN,NaN
2,100000001445,83,0,0,0,66,14,0,7,4,M,zhejiang,NaN
3,100000001575,19,0,0,0,42,22,0,1,0,W,jiangsu,NaN
4,100000001601,7,0,1,0,1,2,0,2,2,NaN,shanghai,NaN


In [24]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# @Date    : 2017-12-25 16:41:30
# @Author  : guanglinzhou (xdzgl812@163.com)

import os
import numpy as np
import pandas as pd
import xgboost as xgb

project_path = '/Users/guanglinzhou/Desktop/DC_Mac/DC_Code/'
train_data = pd.read_csv(project_path + 'preprocess/traindata_output/train_data.csv')
test_data = pd.read_csv(project_path + 'preprocess/testdata_output/test_data.csv')

train_data = train_data.drop(['gender', 'province'], axis=1)
test_data = test_data.drop(['gender', 'province'], axis=1)

train_X = train_data.iloc[:, 1:-1]
train_y = train_data.orderType
test_data_xgb=test_data.iloc[:,1:]

traindata = xgb.DMatrix(train_X, label=train_y)

params = {'booster': 'gbtree',
          'objective': 'binary:logistic',
          'eval_metric': 'auc',
          'gamma': 0.1,
          'min_child_weight': 1.1,
          'max_depth': 5,
          'subsample': 0.7,
          'eta': 0.1,
          'n_estimators': 100
          }

watchlist = [(traindata, 'train')]
print('start training...')
model = xgb.train(params, traindata, num_boost_round=3500, evals=watchlist, early_stopping_rounds=30)
# predict test set
print('start predicting...')
test_data_matrix = xgb.DMatrix(test_data_xgb)
pred = model.predict(test_data_matrix)
class_list = []
for i in pred:
    if (i > 0.5):
        class_list.append(1)
    else:
        class_list.append(0)

test_data['orderType'] = class_list
result = test_data[['userid', 'orderType']]
result.to_csv(project_path + 'result/result1.csv', index=False)


start training...
[0]	train-auc:0.715605
Will train until train-auc hasn't improved in 30 rounds.
[1]	train-auc:0.723002
[2]	train-auc:0.726185
[3]	train-auc:0.734063
[4]	train-auc:0.736438
[5]	train-auc:0.737786
[6]	train-auc:0.739914
[7]	train-auc:0.740549
[8]	train-auc:0.741463
[9]	train-auc:0.743414
[10]	train-auc:0.744912
[11]	train-auc:0.746744
[12]	train-auc:0.748058
[13]	train-auc:0.747914
[14]	train-auc:0.750684
[15]	train-auc:0.751199
[16]	train-auc:0.751879
[17]	train-auc:0.752159
[18]	train-auc:0.752827
[19]	train-auc:0.753982
[20]	train-auc:0.75654
[21]	train-auc:0.757188
[22]	train-auc:0.757482
[23]	train-auc:0.75836
[24]	train-auc:0.759016
[25]	train-auc:0.75958
[26]	train-auc:0.760214
[27]	train-auc:0.762077
[28]	train-auc:0.762494
[29]	train-auc:0.762979
[30]	train-auc:0.764616
[31]	train-auc:0.765444
[32]	train-auc:0.76678
[33]	train-auc:0.76755
[34]	train-auc:0.768155
[35]	train-auc:0.770232
[36]	train-auc:0.771165
[37]	train-auc:0.771646
[38]	train-auc:0.772698
[39]

[331]	train-auc:0.823527
[332]	train-auc:0.823706
[333]	train-auc:0.823742
[334]	train-auc:0.823894
[335]	train-auc:0.824003
[336]	train-auc:0.824112
[337]	train-auc:0.82412
[338]	train-auc:0.824227
[339]	train-auc:0.824275
[340]	train-auc:0.824288
[341]	train-auc:0.824435
[342]	train-auc:0.824489
[343]	train-auc:0.824514
[344]	train-auc:0.824626
[345]	train-auc:0.824615
[346]	train-auc:0.824788
[347]	train-auc:0.824951
[348]	train-auc:0.824999
[349]	train-auc:0.825033
[350]	train-auc:0.825244
[351]	train-auc:0.825323
[352]	train-auc:0.825384
[353]	train-auc:0.825529
[354]	train-auc:0.825596
[355]	train-auc:0.825701
[356]	train-auc:0.825747
[357]	train-auc:0.82585
[358]	train-auc:0.825949
[359]	train-auc:0.826102
[360]	train-auc:0.82612
[361]	train-auc:0.826243
[362]	train-auc:0.826425
[363]	train-auc:0.826523
[364]	train-auc:0.826649
[365]	train-auc:0.826655
[366]	train-auc:0.826806
[367]	train-auc:0.82685
[368]	train-auc:0.827011
[369]	train-auc:0.82711
[370]	train-auc:0.827231
[371]

[661]	train-auc:0.850399
[662]	train-auc:0.850457
[663]	train-auc:0.850482
[664]	train-auc:0.85058
[665]	train-auc:0.850626
[666]	train-auc:0.850636
[667]	train-auc:0.850633
[668]	train-auc:0.850685
[669]	train-auc:0.850752
[670]	train-auc:0.850832
[671]	train-auc:0.850863
[672]	train-auc:0.850951
[673]	train-auc:0.851012
[674]	train-auc:0.851127
[675]	train-auc:0.851241
[676]	train-auc:0.851279
[677]	train-auc:0.851313
[678]	train-auc:0.851371
[679]	train-auc:0.851436
[680]	train-auc:0.851517
[681]	train-auc:0.85153
[682]	train-auc:0.851616
[683]	train-auc:0.851736
[684]	train-auc:0.851771
[685]	train-auc:0.851791
[686]	train-auc:0.851871
[687]	train-auc:0.851925
[688]	train-auc:0.851918
[689]	train-auc:0.851956
[690]	train-auc:0.851976
[691]	train-auc:0.852027
[692]	train-auc:0.852049
[693]	train-auc:0.852171
[694]	train-auc:0.852232
[695]	train-auc:0.852346
[696]	train-auc:0.852405
[697]	train-auc:0.852469
[698]	train-auc:0.852494
[699]	train-auc:0.85251
[700]	train-auc:0.852551
[70

[991]	train-auc:0.867533
[992]	train-auc:0.867568
[993]	train-auc:0.867744
[994]	train-auc:0.867852
[995]	train-auc:0.867869
[996]	train-auc:0.867953
[997]	train-auc:0.868087
[998]	train-auc:0.868086
[999]	train-auc:0.868178
[1000]	train-auc:0.868241
[1001]	train-auc:0.868239
[1002]	train-auc:0.868311
[1003]	train-auc:0.86844
[1004]	train-auc:0.868446
[1005]	train-auc:0.868473
[1006]	train-auc:0.868477
[1007]	train-auc:0.868594
[1008]	train-auc:0.868653
[1009]	train-auc:0.868769
[1010]	train-auc:0.868784
[1011]	train-auc:0.868763
[1012]	train-auc:0.868769
[1013]	train-auc:0.86883
[1014]	train-auc:0.868873
[1015]	train-auc:0.868928
[1016]	train-auc:0.869037
[1017]	train-auc:0.869101
[1018]	train-auc:0.869115
[1019]	train-auc:0.869144
[1020]	train-auc:0.869168
[1021]	train-auc:0.869213
[1022]	train-auc:0.86932
[1023]	train-auc:0.869348
[1024]	train-auc:0.869388
[1025]	train-auc:0.869469
[1026]	train-auc:0.869487
[1027]	train-auc:0.869572
[1028]	train-auc:0.869526
[1029]	train-auc:0.86960

[1308]	train-auc:0.880432
[1309]	train-auc:0.880487
[1310]	train-auc:0.880552
[1311]	train-auc:0.880621
[1312]	train-auc:0.88062
[1313]	train-auc:0.880613
[1314]	train-auc:0.880601
[1315]	train-auc:0.880627
[1316]	train-auc:0.880737
[1317]	train-auc:0.880776
[1318]	train-auc:0.880819
[1319]	train-auc:0.880883
[1320]	train-auc:0.880902
[1321]	train-auc:0.880915
[1322]	train-auc:0.880937
[1323]	train-auc:0.88098
[1324]	train-auc:0.881022
[1325]	train-auc:0.881095
[1326]	train-auc:0.881177
[1327]	train-auc:0.881201
[1328]	train-auc:0.881223
[1329]	train-auc:0.881295
[1330]	train-auc:0.88137
[1331]	train-auc:0.881379
[1332]	train-auc:0.881429
[1333]	train-auc:0.881498
[1334]	train-auc:0.881551
[1335]	train-auc:0.881534
[1336]	train-auc:0.881544
[1337]	train-auc:0.881519
[1338]	train-auc:0.881581
[1339]	train-auc:0.881567
[1340]	train-auc:0.881595
[1341]	train-auc:0.881701
[1342]	train-auc:0.881666
[1343]	train-auc:0.881698
[1344]	train-auc:0.881732
[1345]	train-auc:0.881751
[1346]	train-au

...

[1942]	train-auc:0.89768
[1943]	train-auc:0.897696
[1944]	train-auc:0.897756
[1945]	train-auc:0.897751
[1946]	train-auc:0.897846
[1947]	train-auc:0.897908
[1948]	train-auc:0.897914
[1949]	train-auc:0.897874
[1950]	train-auc:0.897895
[1951]	train-auc:0.897899
[1952]	train-auc:0.897936
[1953]	train-auc:0.897996
[1954]	train-auc:0.898052
[1955]	train-auc:0.898074
[1956]	train-auc:0.898114
[1957]	train-auc:0.898147
[1958]	train-auc:0.898138
[1959]	train-auc:0.89819
[1960]	train-auc:0.898194
[1961]	train-auc:0.898185
[1962]	train-auc:0.898182
[1963]	train-auc:0.898192
[1964]	train-auc:0.898227
[1965]	train-auc:0.898249
[1966]	train-auc:0.898263
[1967]	train-auc:0.898284
[1968]	train-auc:0.898269
[1969]	train-auc:0.898269
[1970]	train-auc:0.898321
[1971]	train-auc:0.898272
[1972]	train-auc:0.898286
[1973]	train-auc:0.898378
[1974]	train-auc:0.898364
[1975]	train-auc:0.89838
[1976]	train-auc:0.898466
[1977]	train-auc:0.898471
[1978]	train-auc:0.898479
[1979]	train-auc:0.898509
[1980]	train-au

[2258]	train-auc:0.904037
[2259]	train-auc:0.904122
[2260]	train-auc:0.90415
[2261]	train-auc:0.904169
[2262]	train-auc:0.90414
[2263]	train-auc:0.904139
[2264]	train-auc:0.904132
[2265]	train-auc:0.904127
[2266]	train-auc:0.904153
[2267]	train-auc:0.904208
[2268]	train-auc:0.904234
[2269]	train-auc:0.904199
[2270]	train-auc:0.904232
[2271]	train-auc:0.904222
[2272]	train-auc:0.904256
[2273]	train-auc:0.904292
[2274]	train-auc:0.904279
[2275]	train-auc:0.90425
[2276]	train-auc:0.904233
[2277]	train-auc:0.904212
[2278]	train-auc:0.904244
[2279]	train-auc:0.904276
[2280]	train-auc:0.904257
[2281]	train-auc:0.904372
[2282]	train-auc:0.904401
[2283]	train-auc:0.904385
[2284]	train-auc:0.904393
[2285]	train-auc:0.9044
[2286]	train-auc:0.904455
[2287]	train-auc:0.90444
[2288]	train-auc:0.904511
[2289]	train-auc:0.904561
[2290]	train-auc:0.904595
[2291]	train-auc:0.904562
[2292]	train-auc:0.904663
[2293]	train-auc:0.904656
[2294]	train-auc:0.904731
[2295]	train-auc:0.904763
[2296]	train-auc:0

[2575]	train-auc:0.909418
[2576]	train-auc:0.909394
[2577]	train-auc:0.909424
[2578]	train-auc:0.90945
[2579]	train-auc:0.909458
[2580]	train-auc:0.909436
[2581]	train-auc:0.909499
[2582]	train-auc:0.909516
[2583]	train-auc:0.909459
[2584]	train-auc:0.909524
[2585]	train-auc:0.909543
[2586]	train-auc:0.909583
[2587]	train-auc:0.909672
[2588]	train-auc:0.909688
[2589]	train-auc:0.909643
[2590]	train-auc:0.909699
[2591]	train-auc:0.909658
[2592]	train-auc:0.909719
[2593]	train-auc:0.909735
[2594]	train-auc:0.90975
[2595]	train-auc:0.909812
[2596]	train-auc:0.909774
[2597]	train-auc:0.909716
[2598]	train-auc:0.909788
[2599]	train-auc:0.909883
[2600]	train-auc:0.909902
[2601]	train-auc:0.909892
[2602]	train-auc:0.909877
[2603]	train-auc:0.909919
[2604]	train-auc:0.909986
[2605]	train-auc:0.909996
[2606]	train-auc:0.910013
[2607]	train-auc:0.910035
[2608]	train-auc:0.910065
[2609]	train-auc:0.910046
[2610]	train-auc:0.910096
[2611]	train-auc:0.9101
[2612]	train-auc:0.910116
[2613]	train-auc

[2892]	train-auc:0.914369
[2893]	train-auc:0.91437
[2894]	train-auc:0.914347
[2895]	train-auc:0.914351
[2896]	train-auc:0.914368
[2897]	train-auc:0.914398
[2898]	train-auc:0.91431
[2899]	train-auc:0.91438
[2900]	train-auc:0.914354
[2901]	train-auc:0.914353
[2902]	train-auc:0.91436
[2903]	train-auc:0.914335
[2904]	train-auc:0.914327
[2905]	train-auc:0.914358
[2906]	train-auc:0.914472
[2907]	train-auc:0.914432
[2908]	train-auc:0.914474
[2909]	train-auc:0.914482
[2910]	train-auc:0.914524
[2911]	train-auc:0.914533
[2912]	train-auc:0.914604
[2913]	train-auc:0.914629
[2914]	train-auc:0.914605
[2915]	train-auc:0.91465
[2916]	train-auc:0.914644
[2917]	train-auc:0.91457
[2918]	train-auc:0.914602
[2919]	train-auc:0.91464
[2920]	train-auc:0.914679
[2921]	train-auc:0.914662
[2922]	train-auc:0.914687
[2923]	train-auc:0.914744
[2924]	train-auc:0.914659
[2925]	train-auc:0.914642
[2926]	train-auc:0.914631
[2927]	train-auc:0.91468
[2928]	train-auc:0.914738
[2929]	train-auc:0.914693
[2930]	train-auc:0.9

[3209]	train-auc:0.918331
[3210]	train-auc:0.91831
[3211]	train-auc:0.918286
[3212]	train-auc:0.918326
[3213]	train-auc:0.918371
[3214]	train-auc:0.918359
[3215]	train-auc:0.918377
[3216]	train-auc:0.918348
[3217]	train-auc:0.918306
[3218]	train-auc:0.918278
[3219]	train-auc:0.918343
[3220]	train-auc:0.91835
[3221]	train-auc:0.918375
[3222]	train-auc:0.918389
[3223]	train-auc:0.918428
[3224]	train-auc:0.918451
[3225]	train-auc:0.918438
[3226]	train-auc:0.918389
[3227]	train-auc:0.918416
[3228]	train-auc:0.918408
[3229]	train-auc:0.918419
[3230]	train-auc:0.918393
[3231]	train-auc:0.918421
[3232]	train-auc:0.918413
[3233]	train-auc:0.918436
[3234]	train-auc:0.918447
[3235]	train-auc:0.918473
[3236]	train-auc:0.918483
[3237]	train-auc:0.918507
[3238]	train-auc:0.918545
[3239]	train-auc:0.918532
[3240]	train-auc:0.918587
[3241]	train-auc:0.918657
[3242]	train-auc:0.91866
[3243]	train-auc:0.918674
[3244]	train-auc:0.918698
[3245]	train-auc:0.91871
[3246]	train-auc:0.918669
[3247]	train-auc